## import packages, define variables

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd # for creation of data frames
import numpy as np #
import seaborn as sns
import nibabel as nib # for loading niftis etc. as arrays
import matplotlib.pyplot as plt # for plotting data
#the output of plotting commands is displayed inline, directly below the code cell that produced it
%matplotlib inline 
import os,glob
from nilearn import plotting, input_data, image #for plotting & working with niftFMis

import pathlib
import pyxnat
import shutil 

###################### update path!! #####################################
base_path = '/root_dir/' ####
##########################################################################

#import qBOLD functions
#import sys
#sys.path.append(os.path.join(base_path + 'scripts/'))
#import mqBOLD_functions as qB


### define FSL directories ######################################################
os.environ["FSLDIR"]='/usr/share/fsl/5.0'
os.environ["FSLOUTPUTTYPE"]='NIFTI_GZ'
os.environ["FSLTCLSH"]='/usr/bin/tclsh'
os.environ["FSLWISH"]='/usr/bin/wish'
os.environ["FSLMULTIFILEQUIT"]="True"
os.environ["LD_LIBRARY_PATH"]='/usr/share/fsl/5.0:/usr/lib/fsl/5.0'


#define ANTS directories
os.environ["ANTSPATH"] = '/opt/ants/bin'
os.environ["PATH"] = f'{os.environ["ANTSPATH"]}:{os.environ["PATH"]}'
if "LD_LIBRARY_PATH" in os.environ:
    os.environ["LD_LIBRARY_PATH"] = f'/opt/ants/lib:{os.environ["LD_LIBRARY_PATH"]}'
else:
    os.environ["LD_LIBRARY_PATH"] = '/opt/ants/lib'
#################################################################################

In [ ]:
data_dir = base_path + 'data/'
derivatives_dir = data_dir + 'derivatives'
raw_dir = data_dir + 'rawdata'
results_dir = base_path + 'results'

sns.set_style("whitegrid")

## N=40
sids = [19, 20, 21, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68]
## N=30
#sids_30 = [19, 20, 21, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55]
## N=10
#sids_10 = [58, 59, 60, 61, 63, 64, 65, 66, 67, 68]

df_participants = pd.read_csv(data_dir + '/participants.tsv',sep='\t')

## Calculate CMRO2 per subject and condition

In [ ]:
### get median of O2Sat across participants ##
#################################################

O2sat_sample = np.zeros((len(sids), 1))
for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    #download & binarize brain mask, store in 'masker' object
    BrMask_CSF = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF.nii')
    mask = image.math_img('img> 0.1', img = BrMask_CSF)
    masker = input_data.NiftiMasker(mask_img=mask)
    
    # download O2 sat & hematocrit values from xnat
    O2sat = df_participants.query('participant_id==@sub')['O2sat'].values
    O2sat_sample[i] = O2sat
    
    
median_O2sat= np.nanmedian(O2sat_sample)
std_O2sat = np.nanstd(O2sat_sample)
print('median O2sat across subject: ' + str(median_O2sat) + ' +/- ' + str(std_O2sat))

In [ ]:
for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    #download & binarize brain mask, store in 'masker' object
    BrMask_CSF = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF.nii')
    mask = image.math_img('img> 0.1', img = BrMask_CSF)
    masker = input_data.NiftiMasker(mask_img=mask)
    
    # download O2 sat & hematocrit values from xnat
    Hk = df_participants.query('participant_id==@sub')['Hct'].values
    O2sat = df_participants.query('participant_id==@sub')['O2sat'].values
        
    if ID < 56: ## here, all 4 conditions were acquired        
        conds = ['rest', 'control', 'mem', 'calc']
        
    if ID > 55:        
        conds = ['control', 'calc'] 
        
    for k, cond in enumerate(conds): 

        #  CBF and OEF niftis f
        CBF = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-' + cond + '_space-T2_cbf.nii')
        CBF_np = masker.fit_transform(CBF)
        OEF = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-' + cond + '_space-T2_oef.nii')
        OEF_np = masker.fit_transform(OEF)
        
        # upscale CBF by 25% because 4 background suppression pulses
        CBF_upscaled = CBF_np / 0.75

        #calculate CaO2 value per subject, O2sat as a fraction
        #
        CaO2 = 0.334 * float(Hk) * 55.6 * float(O2sat)/100;
        
        ## calculate CMRO2 parameter map
        CMRO2 = CBF_upscaled * OEF_np * CaO2 /100;
        
        #store CMRO2 parameter map as nifti
        CMRO2_vol = masker.inverse_transform(CMRO2)
        ## here, CBV was only acquired in control
        if ID < 56:
            CMRO2_vol.to_filename(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-' + cond + '_space-T2_desc-orig_cmro2.nii'))
        ## here, CBV correction was applied: CBV was also acquired in calc
        if ID > 55 and cond == 'calc':
            CMRO2_vol.to_filename(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-' + cond + '_space-T2_desc-CBV_cmro2.nii'))        
        if ID > 55 and cond == 'control':
            CMRO2_vol.to_filename(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-' + cond + '_space-T2_desc-orig_cmro2.nii')) 
            
        ## plot image
        #plotting.plot_img(CMRO2_vol, cmap='jet', colorbar=True, vmin=10, vmax=300, title=sub + ': ' + cond)

## CBV calc-correction

In [ ]:
### create 'new' CBV calc images ###

df_Glasser = pd.read_csv(derivatives_dir + '/CBV_correction_factor.tsv',sep='\t')
correction_factor_percchange = df_Glasser['CBV_percchange']

for i,ID in enumerate(sids_30):
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    ## download CBV control map
    CBV_control = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-T2_cbv.nii')
    CBV_control_img = nib.load(CBV_control)
    CBV_control_arr = np.array(CBV_control_img.dataobj)
    
    ## download Glasser360 parcellation
    Glasser_T2space = os.path.join(derivatives_dir, 'Glasser360_T2space.nii.gz')
    if ID > 23:
        Glasser_T2space = os.path.join(derivatives_dir, 'Glasser360_T2space_35vol.nii.gz')
    Glasser_img=nib.load(Glasser_T2space)
    Glasser_arr=np.array(Glasser_img.dataobj)
       
    ## CBV control img is the baseline, replace voxels that are in Glasser parcels with adjusted CBV values
    CBV_calc_corrected = CBV_control_arr 
    for r, region in enumerate(range(1, 361)): 
        correction_factor = correction_factor_percchange/100 ##from percent to ratio
        CBV_calc_corrected[Glasser_arr==region] = CBV_control_arr[Glasser_arr==region] + (correction_factor[r]*CBV_control_arr[Glasser_arr==region])
    

    ## turn into nifti, display
    CBV_calc_corrected_img = nib.Nifti1Image(CBV_calc_corrected, CBV_control_img.affine)
    T1w_T2 = os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-T2_T1w.nii')

    #plotting.plot_img(CBV_calc_corrected_img,bg_img=T1w_T2, cmap='jet', colorbar=True, threshold=0.1, vmax=10, black_bg=True, 
    #                             title = 'CBV calc corrected for sub ' + subj_id)
    #plotting.plot_img(CBV_calc_corrected_img, bg_img=T1w_T2, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=10, threshold=0.1,
    #               title = 'CBV calc corrected for sub ' + subj_id)
    #plotting.plot_img(CBV_control_img, bg_img=T1w_T2, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=10, threshold=0.1,
    #               title = 'CBV control for sub ' + subj_id)
    
    ## save 
    nib.save(CBV_calc_corrected_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_cbv.nii'))


In [ ]:
### recalculate OEF and CMRO2 for calc ###

for i,ID in enumerate(sids_30):
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    
    # download O2 sat & hematocrit values from xnat
    Hct = df_participants.query('participant_id==@sub')['Hct'].values
    O2sat = df_participants.query('participant_id==@sub')['O2sat'].values
        
    ##############################
    ## calculate adjusted rOEF ##
    #############################
    
    
    R2prime_calc =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_R2prime.nii')
    R2prime_calc_nii = nib.load(R2prime_calc)
    R2prime_calc_arr = np.array(R2prime_calc_nii.dataobj)
    
    ## co-register CBV and T2, do not have the same number of slices for ID<26! 
    CBV_calc_corrected =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_cbv.nii')
    CBV_calc_corrected_nii = nib.load(CBV_calc_corrected)
    CBV_calc_corrected_arr = np.array(CBV_calc_corrected_nii.dataobj)
    
    BrMask_CSF = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF.nii')
    BrMsk_CSF_nii = nib.load(BrMsk_CSF)
    BrMsk_CSF_arr = np.array(BrMsk_CSF_nii.dataobj)
      
    ## here, the brain mask has too many slices
    T1w_T2 = os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-T2_T1w.nii')
    T1w_T2_brain = os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-T2_desc-brain_T1w.nii.gz')
    ! bet {T1w_T2} {T1w_T2_brain}
    BrMask_CSF_sliced = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF_resliced.nii.gz')
    ! flirt -in {BrMask_CSF} -ref {T1w_T2_brain} -out {BrMask_CSF_sliced} -dof 6 
    plotting.plot_img(BrMask_CSF_sliced, bg_img=T1w_T2, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=350, vmin = 10, threshold=0.1,
                   title = 'CMRO2 calc corrected: ' + sub)
    
    BrMsk_CSF_nii = nib.load(BrMask_CSF_sliced)
    BrMsk_CSF_arr = np.array(BrMsk_CSF_nii.dataobj)

        
    mask = BrMsk_CSF_arr>0.5;
    
    # [Hz], physiologic constant derived from literature
    # 0.85: factor for small-vessel hematocrit
    C = 4/3*267.61918*np.pi*0.264*(Hct/100)*0.85*3; # 317 Hz

    rCBV = CBV_calc_corrected_arr/100.0; #CBV fraction, e.g. CBV_WM = 0.015
    rCBV[rCBV < 0] = 0;
    eps=2.2204e-16 #to not divide by zero
    rOEF = (R2prime_calc_arr / (C*rCBV+eps)) * mask

    # reasonable boundary to avoid unreasonably high hot spots
    rOEFmax_DSC = 5*np.nanmedian(rOEF[mask]);
    if rOEFmax_DSC < 1.5:
            rOEFmax_DSC = 1.5 # FIXED Threshold
    rOEF[rOEF > rOEFmax_DSC] = rOEFmax_DSC
    rOEF = rOEF *mask

    rOEF_calc_corrected_img = nib.Nifti1Image(rOEF, CBV_calc_corrected_nii.affine)
    nib.save(rOEF_calc_corrected_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_desc-CBV_oef.nii'))


    ##############################
    ## calculate adjusted CMRO2 ##
    ##############################
    
    
    # download CSF and OEF niftis from xnat
    CBF = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_cbf.nii')
    CBF_nii = nib.load(CBF)
    CBF_np= np.array(CBF_nii.dataobj)
    OEF_corrected = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_desc-CBV_oef.nii')
    OEF_nii = nib.load(OEF_corrected)
    OEF_np= np.array(OEF_nii.dataobj)
    
    # upscale CSF by 25% because of scanner
    CBF_upscaled = CBF_np / 0.75 # upscale CSF by 25% because of scanner

    #calculate CaO2 value per subject, O2sat as a fraction
    CaO2 = 0.334 * Hct * 55.6 * float(O2sat)/100;
        
    ## calculate CMRO2 parameter map
    CMRO2 = CBF_upscaled * OEF_np * CaO2 /100;
    CMRO2 = CMRO2 * mask
        
    #store CMRO2 parameter map as nifti
    CMRO2_calc_corrected_img = nib.Nifti1Image(CMRO2, CBV_calc_corrected_nii.affine)

    nib.save(CMRO2_calc_corrected_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_desc-CBV_cmro2.nii.gz'))

    ## plot output
    T1w_T2 = os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-T2_T1w.nii')
    plotting.plot_img(CMRO2_calc_corrected_img, bg_img=T1w_T2, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=400, vmin = 10, threshold=0.1,
                   title = 'CMRO2 calc corrected: ' + sub)
    CMRO2_calc_img = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_desc-orig_cmro2.nii')
    plotting.plot_img(CMRO2_calc_img, bg_img=T1w_T2, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=400, vmin = 10, threshold=0.1,
                   title = 'CMRO2 orig: ' + sub)

### Re-calcualate OEF and CMRO2 based on CBF lowres: CBF Lowres * OEF Lowres (re-create OEF based on R2' and CBV, then re-calculate CMRO2 lowres)

In [ ]:
## Co-register all parameter maps to CBF control Lowres
parameters = ['R2prime', 'oef', 'cbv']

conds_cbv = ['control', 'calc']


for i,ID in enumerate(sids):
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    if ID < 56: ## here, all 4 conditions were acquired        
        conds = ['rest', 'control', 'mem', 'calc']
        
    if ID > 55:        
        conds = ['control', 'calc'] 
        
    ##input
    T1w_T2space = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-T2_T1w.nii')
    T1w_T2space_brain = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-T2_T1w_brain.nii.gz')
    
    CBF_lowres_control = os.path.join(derivatives_dir, sub, 'qmri', sub + '_task-control_space-lowres_cbf.nii.gz')

    ## first, generate transform from T1w T2 space to CBF lowres ()
    T1w_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-lowres_T1w.nii.gz')
    T1w_lowres_brain = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-lowres_T1w_brain.nii.gz')
    T2_to_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-T2_to_space-lowres.mat')
    BrMsk_CSF_lowres = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF_space-lowres.nii.gz')
    
    ! flirt -in {T1w_T2space_brain} -ref {CBF_lowres_ctrl} -omat {T2_to_lowres} -out {T1w_lowres_brain} -dof 6 
    plotting.plot_img(CBF_lowres_control, bg_img=T1w_lowres_brain, threshold = 10, cmap='jet', title= sub +': CBF on T1w lowres')    
    
    ## coregister all CBF images to CBF control lowres
    if ID < 56: ## here, all 4 conditions were acquired        
        conds_cbf = ['rest', 'mem', 'calc']  
    if ID > 55:        
        conds_cbf = ['calc'] 
    
    ## coregister all CBF lowres images to CBF lowres control
    for k, cond in enumerate(conds_cbf): 
        #os.remove(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbf.nii'))
        CBF_lowres_orig= os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbf_orig.nii')
        if os.path.exists(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbf.nii')):
            os.remove(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbf.nii'))
        CBF_lowres= os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbf.nii.gz')
        ! flirt -in {CBF_lowres_orig} -ref {CBF_lowres_control} -out {CBF_lowres} -dof 6 
        plotting.plot_img(CBF_lowres, bg_img=T1w_lowres_brain, threshold = 10, vmax=50, cmap='jet', title= sub + cond +': CBF lowres')    

        
    for par in parameters:
        if par !='cbv':
            for k, cond in enumerate(conds): 
                par_T2 = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-T2_'+par+'.nii')
                par_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_'+par+'.nii.gz')

                if cond == 'calc' and par == 'oef':
                    par_T2 = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-T2_desc-CBV_oef.nii.gz')
                    par_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_desc-CBV_oef.nii.gz')
                    if ID > 55: 
                        par_T2 = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-T2_oef.nii.gz')

                ! flirt -in {par_T2} -ref {T1w_lowres_brain} -applyxfm -init {T2_to_lowres} -out {par_lowres} -dof 6 
                plotting.plot_img(par_lowres, bg_img=T1w_lowres_brain, threshold = 0.2, cmap='jet', title= sub + cond + par +': lowres')
            
            
        ## cbv    
        if par == 'cbv':
            for k, cond in enumerate(conds_cbv): 
                par_T2 = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-T2_'+par+'.nii')
                par_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_'+par+'.nii.gz')

                ! flirt -in {par_T2} -ref {T1w_lowres_brain} -applyxfm -init {T2_to_lowres} -out {par_lowres} -dof 6 
                plotting.plot_img(par_lowres, bg_img=T1w_lowres_brain, threshold = 0.2, cmap='jet', title= sub + cond + par +': lowres')

In [ ]:
## REcalculate CMRO2 based on lowres R2prime and CBV

for i,ID in enumerate(sids):
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    
    if ID < 56: ## here, all 4 conditions were acquired        
        conds = ['rest', 'control', 'mem', 'calc']
        
    if ID > 55:        
        conds = ['control', 'calc'] 
    
    
    ##input
    T1w_lowres_brain = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-lowres_T1w_brain.nii.gz')
    T2_to_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-T2_to_space-lowres.mat')
    T1w_T2space = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-T2_T1w.nii')
    CBF_lowres_control = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-lowres_cbf.nii.gz')
    
    #download, binarize brain mask and coregister to CBF lowres, store in 'masker' object
    BrMask_CSF = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF.nii')
    BrMask_CSF_lowres = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF_space-lowres.nii.gz')    
    ! flirt -in {BrMask_CSF} -ref {T1w_lowres_brain} -applyxfm -init {T2_to_lowres} -out {BrMask_CSF_lowres} -dof 6 -interp nearestneighbour
    plotting.plot_img(BrMask_CSF_lowres, bg_img=T1w_lowres_brain, threshold=0.1, cmap='jet', title= sub  +': brain mask')
    
    BrMask_CSF_lowres_arr = np.array(nib.load(BrMask_CSF_lowres).dataobj)
    mask = BrMask_CSF_lowres_arr>0.5;
    
    ## create nii mask
    BrMask_CSF_lowres_arr_mask = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF_space-lowres_mask.nii.gz')  
    ! fslmaths {BrMask_CSF_lowres} -thr 0.5 -bin {BrMask_CSF_lowres_arr_mask}
    masker = input_data.NiftiMasker(mask_img=BrMask_CSF_lowres_arr_mask)
    
    # download O2 sat & hematocrit values from xnat
    Hk = df_participants.query('participant_id==@sub')['Hct'].values
    O2sat = df_participants.query('participant_id==@sub')['O2sat'].values
    
    
##Re-calculate CMRO2 ####
########################################
    for cond in conds: 

    ##############################
    ## calculate rOEF ##
    #############################  
    
        R2prime =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_R2prime.nii.gz')
        R2prime_nii = nib.load(R2prime)
        R2prime_arr = np.array(R2prime_nii.dataobj)

        ## CBV (= corrected for calc)
        if cond == 'calc':
            CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbv.nii.gz')
        if cond != 'calc': ## then use control image!
            CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-lowres_cbv.nii.gz')
        CBV_nii = nib.load(CBV)
        CBV_arr = np.array(CBV_nii.dataobj)

    
        # [Hz], physiologic constant derived from literature
        # 0.85: factor for small-vessel hematocrit
        C = 4/3*267.61918*np.pi*0.264*(Hk/100)*0.85*3; # 317 Hz

        rCBV = CBV_arr/100.0; #CBV fraction, e.g. CBV_WM = 0.015
        rCBV[rCBV < 0] = 0;
        eps=2.2204e-16 #to not divide by zero
        rOEF = (R2prime_arr / (C*rCBV+eps)) * mask

        # reasonable boundary to avoid unreasonably high hot spots
        rOEFmax_DSC = 5*np.nanmedian(rOEF[mask]);
        if rOEFmax_DSC < 1.5:
                rOEFmax_DSC = 1.5 # FIXED Threshold
        rOEF[rOEF > rOEFmax_DSC] = rOEFmax_DSC
        rOEF = rOEF *mask

        rOEF_img = nib.Nifti1Image(rOEF, CBV_nii.affine)
        if cond == 'calc':
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_desc-CBV_oef.nii.gz'))
        if cond != 'calc':
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_oef.nii.gz'))
            
        CBF_lowres= os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbf.nii.gz')

        if cond == 'calc':
            OEF_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_desc-CBV_oef.nii.gz')
        if cond != 'calc':
            OEF_lowres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_oef.nii.gz')

        plotting.plot_img(CBF_lowres, bg_img=T1w_lowres_brain, threshold=0.1, cmap='jet', title= sub + cond +': CBF lowres')
        plotting.plot_img(OEF_lowres, bg_img=T1w_lowres_brain, threshold=0.01, cmap='jet', title= sub + cond +': OEF lowres')

        #  CBF and OEF niftis f
        CBF_np = masker.fit_transform(CBF_lowres)
        OEF_np = masker.fit_transform(OEF_lowres)

        # upscale CBF by 25% because 4 background suppression pulses
        CBF_upscaled = CBF_np / 0.75

        #calculate CaO2 value per subject, O2sat as a fraction
        #
        CaO2 = 0.334 * float(Hk) * 55.6 * float(O2sat)/100;

        ## calculate CMRO2 parameter map
        CMRO2 = CBF_upscaled * OEF_np * CaO2 /100;

        #store CMRO2 parameter map as nifti
        CMRO2_vol = masker.inverse_transform(CMRO2)
        ## here, CBV was only acquired in control
        if cond == 'calc':
            CMRO2_vol.to_filename(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-' + cond + '_space-lowres_desc-CBV_cmro2.nii.gz'))
        if cond != 'calc':
            CMRO2_vol.to_filename(os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-' + cond + '_space-lowres_desc-orig_cmro2.nii.gz')) 

        plotting.plot_img(CMRO2_vol, bg_img=T1w_lowres_brain, threshold = 10, vmin=20, vmax=350, cmap='jet', title= sub + cond +': CMRO2 lowres')
 

In [ ]:
## normalize to MNI 2mm space!
parameters=['R2prime','cmro2', 'cbf','oef', 'cbv'] ##all these parameters get normalized from native, lowres space!

for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    sub_dir =  os.path.join(data_dir, sub)
    dir_deriv = os.path.join(data_dir, 'derivatives', sub)

   
    T1w_lowres_brain = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_space-lowres_T1w_brain.nii.gz')
    T1w=os.path.join(dir_deriv, 'anat', sub + '_desc-fmriprep_T1w.nii.gz')
    T1w_brain=os.path.join(dir_deriv, 'anat', sub + '_desc-fmriprep_T1w_brain.nii.gz')
    transform_to_standard=os.path.join(dir_deriv, 'anat', sub + '_from-T1w_to-MNI152NLin6Asym_mode-image_xfm.h5')
    T1w_MNIspace=os.path.join(dir_deriv, 'anat', sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz')
    
    ##define output: first, lowres to T1w
    Lowres_to_T1w=os.path.join(dir_deriv, 'qmri', sub + '_Lowres_to_T1w.mat')    

    ! flirt -in {T1w_lowres_brain} -ref {T1w_brain} -omat {Lowres_to_T1w} -dof 6
    
    ## coregister brain mask to MNI space
    BrMsk_CSF_T1space = os.path.join(derivatives_dir, sub, 'anat',  sub + '_desc-fmriprep_brain_mask.nii.gz')
    BrMsk_CSF_MNIlowres =  os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF_space-MNI152-lowres.nii.gz')
    ! antsApplyTransforms -d 3 --float 1 --verbose 1 -i {BrMsk_CSF_T1space} -r {T1w_MNIspace} -t {transform_to_standard} -o {BrMsk_CSF_MNIlowres} 

    #plotting.plot_img(BrMsk_CSF_MNIlowres, bg_img = T1w_MNIspace, display_mode='z', cmap='jet', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0.1, black_bg=True, title = sub + ': ' +  par + cond + ' MNI ')
    
    for p, par in enumerate(parameters):
        print(par)
        if ID < 56 and par !='cbv':
            conds = ['rest', 'control', 'mem', 'calc']
        if ID > 55 or par == 'cbv':
            conds = ['control', 'calc']   
    
        
        for cond in conds:
            print(cond)
        
            if (par =='cbf') or par == 'R2prime': 
                parameter_map=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-lowres_'+par+'.nii.gz')            
                out_T1w=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T1w-lowres_'+par+'.nii.gz')
                out_MNI=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-MNI152-lowres_'+par+'.nii.gz')
                #plotting.plot_img(parameter_map, bg_img = T1w_lowres_brain, display_mode='z', cmap='jet', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0.1, black_bg=True, title = sub + ': ' +  par +  ' MNI')
                
            if (par =='cmro2') or par == 'oef' :
                if cond == 'calc':
                    parameter_map = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_desc-CBV_'+par+'.nii.gz')
                    out_T1w=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T1w-lowres_desc-CBV_'+par+'.nii.gz')
                    out_MNI=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-MNI152-lowres_desc-CBV_'+par+'.nii.gz')
                if cond != 'calc':
                    if par == 'oef':
                        parameter_map = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_'+par+'.nii.gz')
                        out_T1w=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T1w-lowres_desc-orig_'+par+'.nii.gz')
                        out_MNI=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-MNI152-lowres_desc-orig_'+par+'.nii.gz')
                    if par == 'cmro2':
                        parameter_map = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_desc-orig_'+par+'.nii.gz')
                        out_T1w=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T1w-lowres_desc-orig_'+par+'.nii.gz')
                        out_MNI=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-MNI152-lowres_desc-orig_'+par+'.nii.gz')
                    
            if par == 'cbv':
                if cond == 'calc' or cond == 'control':
                    parameter_map = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_'+par+'.nii.gz')
                    out_T1w=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T1w-lowres_'+par+'.nii.gz')
                    out_MNI=os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-MNI152-lowres_'+par+'.nii.gz')
                        
            ##apply transformation from lowres to highres T1 space for parameter maps
            ! flirt -in {parameter_map} -ref {T1w_brain} -applyxfm -init {Lowres_to_T1w} -out {out_T1w} -dof 6
            #plotting.plot_img(out_T1w, bg_img = T1w, display_mode='z', cmap='jet', cut_coords = ( 30, 40, 50, 60, 70, 80, 90), threshold = 0.1, black_bg=True, title = sub + ': ' +  par + cond + ' T1w ')
            
            ##apply transformation (saved by fmriprep) from T1 to standard space
            ! antsApplyTransforms -d 3 --float 1 --verbose 1 -i {out_T1w} -r {T1w_MNIspace} -t {transform_to_standard} -o {out_MNI} 

            ## plot
            plotting.plot_img(out_MNI, bg_img = T1w_MNIspace, display_mode='z', cmap='jet', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0.1, black_bg=True, title = sub + ': ' +  par + cond + ' MNI ')
            

### store BOLD percchange niftis

In [ ]:
## coregister func to T2 map
for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    sub_ID = "p{:03d}".format(ID) 
    
    T1w_T1space = os.path.join(derivatives_dir, sub, 'anat',sub + '_desc-fmriprep_T1w.nii.gz')
    T1w_T2space = os.path.join(derivatives_dir, sub, 'qmri',sub + '_space-T2_T1w.nii')
    T1w_T2space_brain = os.path.join(derivatives_dir, sub, 'qmri',sub + '_space-T2_T1w_brain.nii.gz')
    T1w_to_T2space = os.path.join(derivatives_dir, sub, 'qmri',sub + '_T1w_to_T2space.mat')
    
    func_T1space = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-all_space-T1w_desc-preproc_bold.nii.gz')    
    func_T2space = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-all_space-T2_desc-preproc_bold.nii.gz')
    
    ## brain-extract T1w T2 space
    ! bet {T1w_T2space} {T1w_T2space_brain} -f 0.3
    plotting.plot_img(T1w_T2space_brain,  bg_img =T1w_T2space, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 1, black_bg=True, title = 'T1w_brain : ' + sub)
    
    ## coregister T1 to T2 space 
    ! epi_reg --epi={func_T1space} --t1={T1w_T2space} --t1brain={T1w_T2space_brain} --out={func_T2space}

    
    func_T2space_slice = image.index_img(func_T2space, 0) ## first volume
    plotting.plot_img(func_T2space_slice,  bg_img =T1w_T2space, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 10000, black_bg=True, title = 'func : ' + sub)
    

In [ ]:
for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    
    if ID < 56:             
        contrasts = ['calccontrol', 'memcontrol', 'calcrest']
        #contrasts = ['memcontrol']
                    
        func_T2space = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-all_space-T2_desc-preproc_bold.nii.gz')
        func_img = nib.load(func_T2space)
        func_arr = np.array(func_img.dataobj) 

        ##taking HRF into account, i.e. only take time-points after 5TR = 7sec
        rest = func_arr[:, :, :,list(range(5, 25)) + list(range(130, 150)) + list(range(230, 250)) + list(range(330, 350))] #rest
        baseline_rest = np.nanmedian(rest, axis=3) #median across time-points

        control = func_arr[:, :, :, list(range(80, 100)) + list(range(205, 225)) + list(range(280, 300)) + list(range(355, 375))]
        baseline_control = np.nanmedian(control, axis=3) #median across time-points

                
        ## get %change per time-point
        percchange_control=np.zeros((func_arr.shape))
        percchange_rest=np.zeros((func_arr.shape))
        percchange_mem = np.zeros((func_arr.shape))
        for i in range(0, func_arr.shape[3]):
            percchange_control[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_control) / baseline_control
            percchange_rest[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_rest) / baseline_rest

        for l, con in enumerate(contrasts): 

            if con == 'calccontrol':
                percchange = np.nanmedian(percchange_control[:, :, :,list(range(55, 75)) + list(range(155, 175)) + list(range(255, 275)) + list(range(305, 325))], axis=3)

            if con == 'memcontrol':
                percchange = np.nanmedian(percchange_control[:, :, :,list(range(30, 50)) + list(range(105, 125)) + list(range(180, 200)) + list(range(380, 400))], axis=3)

            if con == 'calcrest':
                percchange = np.nanmedian(percchange_rest[:, :, :,list(range(55, 75)) + list(range(155, 175)) + list(range(255, 275)) + list(range(305, 325))], axis=3)

            ## save nifti
            T1w_T2 = nib.load(os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-T2_T1w.nii'))
            percchange_img = nib.Nifti1Image(percchange, T1w_T2.affine)
            plotting.plot_img(percchange_img,  cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), colorbar=True, black_bg=True, threshold=0, vmin=0, vmax=5, title = 'percchange : ' + sub)

            nib.save(percchange_img, os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-T2_BOLD_percchange.nii.gz'))


    if ID > 55:
        con = 'calccontrol'
        
        func_T2space = os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-all_space-T2_desc-preproc_bold.nii.gz')
        func_img = nib.load(func_T2space)
        func_arr = np.array(func_img.dataobj)             

        ##taking HRF into account, i.e. only take time-points after 5TR = 7sec
        control = func_arr[:, :, :, list(range(5, 25)) + list(range(55, 75)) + list(range(105, 125)) + list(range(155, 175))]
        baseline_control = np.nanmedian(control, axis=3) #median across time-points

        ## get %change per time-point
        percchange_control=np.zeros((func_arr.shape))
        for i in range(0, func_arr.shape[3]):
            percchange_control[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_control) / baseline_control

        percchange = np.nanmedian(percchange_control[:, :, :,list(range(30, 50)) + list(range(80, 100)) + list(range(130, 150)) + list(range(180, 200))], axis=3)

        ## save nifti
        T1w_T2 = nib.load(os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-T2_T1w.nii'))
        percchange_img = nib.Nifti1Image(percchange, T1w_T2.affine)
        nib.save(percchange_img, os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-T2_BOLD_percchange.nii.gz'))

### percchange downsampled to CBF lowres and normalized to MNI space


In [ ]:
for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    sub_ID = "p{:03d}".format(ID) 
    
    ####################################
    ## coregister func to CBF lowres ##
    ###################################
    
    CBF_lowres_baseline = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-lowres_cbf.nii')
    
    T1w_T1space = os.path.join(derivatives_dir, sub, 'anat',sub + '_desc-fmriprep_T1w.nii.gz')
    T1w_T1space_brain = os.path.join(derivatives_dir, sub, 'anat',sub + '_desc-fmriprep_T1w_brain.nii.gz')
    T1w_lowres = os.path.join(derivatives_dir, sub, 'anat',sub + '_desc-fmriprep_lowres.nii.gz')
    T1w_lowres_brain = os.path.join(derivatives_dir, sub, 'anat',sub + '_desc-fmriprep_lowres_brain.nii.gz')
    T1w_to_lowres = os.path.join(derivatives_dir, sub, 'qmri',sub + '_T1w_to_lowres.mat')
    
    ## create T1w lowres
    ! flirt -in {T1w_T1space_brain} -ref {CBF_lowres_baseline} -omat {T1w_to_lowres} -dof 12
    ! flirt -in {T1w_T1space} -ref {CBF_lowres_baseline} -applyxfm -init {T1w_to_lowres}  -out {T1w_lowres} -dof 12 
    plotting.plot_img(T1w_lowres_brain,  bg_img =T1w_T1space, cmap='Greys',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 1, black_bg=True, title = 'T1w lowres : ' + sub)
    
    func_T1space = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-all_space-T1w_desc-preproc_bold.nii.gz')    
    func_lowres = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-all_space-lowres_desc-preproc_bold.nii.gz')    
    
    
    ## coregister T1 to CBF lowres space 
    ! bet {T1w_lowres} {T1w_lowres_brain} -f 0.3    
    ! epi_reg --epi={func_T1space} --t1={T1w_lowres} --t1brain={T1w_lowres_brain} --out={func_lowres}
    
    func_lowres_slice = image.index_img(func_lowres, 0) ## first volume
    plotting.plot_img(func_lowres_slice,  bg_img =T1w_T1space, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 10000, black_bg=True, title = 'func lowres : ' + sub)
    
    ##########################
    ## create %change lowres ##
    ##########################
    if ID < 56:
        contrasts = ['calccontrol', 'memcontrol', 'calcrest']
        contrasts = ['calccontrol']
                    
        func_lowres = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-all_space-lowres_desc-preproc_bold.nii.gz')    
        func_img = nib.load(func_lowres)
        func_arr = np.array(func_img.dataobj) 

        ##taking HRF into account, i.e. only take time-points after 5TR = 7sec
        rest = func_arr[:, :, :,list(range(5, 25)) + list(range(130, 150)) + list(range(230, 250)) + list(range(330, 350))] #rest
        baseline_rest = np.nanmedian(rest, axis=3) #median across time-points

        control = func_arr[:, :, :, list(range(80, 100)) + list(range(205, 225)) + list(range(280, 300)) + list(range(355, 375))]
        baseline_control = np.nanmedian(control, axis=3) #median across time-points
                
        ## get %change per time-point
        percchange_control=np.zeros((func_arr.shape))
        percchange_rest=np.zeros((func_arr.shape))
        percchange_mem = np.zeros((func_arr.shape))
        for i in range(0, func_arr.shape[3]):
            percchange_control[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_control) / baseline_control
            percchange_rest[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_rest) / baseline_rest

        for l, con in enumerate(contrasts): 

            if con == 'calccontrol':
                percchange = np.nanmedian(percchange_control[:, :, :,list(range(55, 75)) + list(range(155, 175)) + list(range(255, 275)) + list(range(305, 325))], axis=3)

            if con == 'memcontrol':
                percchange = np.nanmedian(percchange_control[:, :, :,list(range(30, 50)) + list(range(105, 125)) + list(range(180, 200)) + list(range(380, 400))], axis=3)

            if con == 'calcrest':
                percchange = np.nanmedian(percchange_rest[:, :, :,list(range(55, 75)) + list(range(155, 175)) + list(range(255, 275)) + list(range(305, 325))], axis=3)

            ## save nifti
            T1w_lowres_brain_nii = nib.load(T1w_lowres_brain)
            percchange_img = nib.Nifti1Image(percchange, T1w_lowres_brain_nii.affine)
            nib.save(percchange_img, os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-lowres_BOLD_percchange.nii.gz'))
            plotting.plot_img(percchange_img, bg_img =T1w_T1space, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), colorbar=True, black_bg=True, threshold=0.5, vmin=-2, vmax=2, title = 'percchange : ' + sub)

    if ID > 55:
        con = 'calccontrol'
        
        func_lowres = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-all_space-lowres_desc-preproc_bold.nii.gz')    
        func_img = nib.load(func_lowres)
        func_arr = np.array(func_img.dataobj)             

        ##taking HRF into account, i.e. only take time-points after 5TR = 7sec
        control = func_arr[:, :, :, list(range(5, 25)) + list(range(55, 75)) + list(range(105, 125)) + list(range(155, 175))]
        baseline_control = np.nanmedian(control, axis=3) #median across time-points

        ## get %change per time-point
        percchange_control=np.zeros((func_arr.shape))
        for i in range(0, func_arr.shape[3]):
            percchange_control[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_control) / baseline_control

        percchange = np.nanmedian(percchange_control[:, :, :,list(range(30, 50)) + list(range(80, 100)) + list(range(130, 150)) + list(range(180, 200))], axis=3)

        ## save nifti
        T1w_lowres_brain_nii = nib.load(T1w_lowres_brain)
        percchange_img = nib.Nifti1Image(percchange, T1w_lowres_brain_nii.affine)
        nib.save(percchange_img, os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-lowres_BOLD_percchange.nii.gz'))    
        plotting.plot_img(percchange_img, bg_img = T1w_T1space, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), colorbar=True, black_bg=True, threshold=0.5, vmin=-2, vmax=2, title = 'percchange : ' + sub)
    
    ##########################
    ## normalize percchange to MNI 2mm ##
    ##########################
    T1w_MNIspace=os.path.join(derivatives_dir, sub, 'anat', sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz')

    transform_to_standard=os.path.join(derivatives_dir, sub,'anat', sub + '_from-T1w_to-MNI152NLin6Asym_mode-image_xfm.h5')
    func_lowres_to_highres = os.path.join(derivatives_dir, sub,'func',  sub + '_func_lowres_to_highres_T1.mat')

    for l, con in enumerate(contrasts): 
        out_MNI = os.path.join(derivatives_dir, sub,'func',  sub + '_task-'+con+'_space-MNI152-lowres_desc-fmriprep_BOLD_percchange.nii.gz')
        percchange_lowres = os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-lowres_BOLD_percchange.nii.gz' )  
        percchange_highres = os.path.join(derivatives_dir, sub, 'func',  sub + '_task-'+con+'_space-highres_BOLD_percchange.nii.gz' )   

        ! flirt -in {T1w_lowres_brain} -ref {T1w_T1space_brain} -omat {func_lowres_to_highres} -dof 6
        ! flirt -in {percchange_lowres} -ref {T1w_T1space_brain} -applyxfm -init {func_lowres_to_highres} -out {percchange_highres} -dof 6

        ##apply transformation (saved by fmriprep) from T1 to standard space
        ! antsApplyTransforms -d 3 -e 3 --float 1 --verbose 1 -i {percchange_highres} -r {T1w_MNIspace} -t {transform_to_standard} -o {out_MNI} 

        ## plot
        plotting.plot_img(out_MNI, bg_img = T1w_MNIspace, display_mode='z', cmap='jet', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0.5, vmin=-2, vmax=2, black_bg=True, title = sub + ': percchange MNI 2mm lowres')        

## BOLD percchange MNI space based on fmriprep output

In [ ]:
for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    
    if ID < 56:             
        contrasts = ['calccontrol', 'memcontrol', 'calcrest']
        #contrasts = ['calcrest']
                    
        func_MNIspace = os.path.join(derivatives_dir, sub, 'func',  sub +  'task-all_space-MNI152_res-2_desc-preproc_bold.nii.gz')
        func_img = nib.load(func_MNIspace)
        func_arr = np.array(func_img.dataobj) 

        ##taking HRF into account, i.e. only take time-points after 5TR = 7sec
        rest = func_arr[:, :, :,list(range(5, 25)) + list(range(130, 150)) + list(range(230, 250)) + list(range(330, 350))] #rest
        baseline_rest = np.nanmedian(rest, axis=3) #median across time-points

        control = func_arr[:, :, :, list(range(80, 100)) + list(range(205, 225)) + list(range(280, 300)) + list(range(355, 375))]
        baseline_control = np.nanmedian(control, axis=3) #median across time-points

                
        ## get %change per time-point
        percchange_control=np.zeros((func_arr.shape))
        percchange_rest=np.zeros((func_arr.shape))
        percchange_mem = np.zeros((func_arr.shape))
        for i in range(0, func_arr.shape[3]):
            percchange_control[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_control) / baseline_control
            percchange_rest[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_rest) / baseline_rest

        for l, con in enumerate(contrasts): 

            if con == 'calccontrol':
                percchange = np.nanmedian(percchange_control[:, :, :,list(range(55, 75)) + list(range(155, 175)) + list(range(255, 275)) + list(range(305, 325))], axis=3)

            if con == 'memcontrol':
                percchange = np.nanmedian(percchange_control[:, :, :,list(range(30, 50)) + list(range(105, 125)) + list(range(180, 200)) + list(range(380, 400))], axis=3)

            if con == 'calcrest':
                percchange = np.nanmedian(percchange_rest[:, :, :,list(range(55, 75)) + list(range(155, 175)) + list(range(255, 275)) + list(range(305, 325))], axis=3)

            ## save nifti
            T1w_MNI = nib.load(os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz'))
            percchange_img = nib.Nifti1Image(percchange, T1w_MNI.affine)
            #plotting.plot_img(percchange_img,  cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), colorbar=True, black_bg=True, threshold=0, vmin=0, vmax=5, title = 'percchange : ' + sub)

            nib.save(percchange_img, os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-MNI152_desc-fmriprep_BOLD_percchange.nii.gz'))


    if ID > 55:
        con = 'calccontrol'
        
        func_MNIspace = os.path.join(derivatives_dir, sub, 'func',  sub +  'task-all_space-MNI152_res-2_desc-preproc_bold.nii.gz')
        func_img = nib.load(func_MNIspace)
        func_arr = np.array(func_img.dataobj)             

        ##taking HRF into account, i.e. only take time-points after 5TR = 7sec
        control = func_arr[:, :, :, list(range(5, 25)) + list(range(55, 75)) + list(range(105, 125)) + list(range(155, 175))]
        baseline_control = np.nanmedian(control, axis=3) #median across time-points

        ## get %change per time-point
        percchange_control=np.zeros((func_arr.shape))
        for i in range(0, func_arr.shape[3]):
            percchange_control[:, :, :, i] = 100*(func_arr[:, :, :, i] - baseline_control) / baseline_control

        percchange = np.nanmedian(percchange_control[:, :, :,list(range(30, 50)) + list(range(80, 100)) + list(range(130, 150)) + list(range(180, 200))], axis=3)

        ## save nifti
        T1w_MNI = nib.load(os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz'))
        percchange_img = nib.Nifti1Image(percchange, T1w_MNI.affine)
        nib.save(percchange_img, os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-MNI152_desc-fmriprep_BOLD_percchange.nii.gz'))

### semi-quantitative CMRO2

In [ ]:
import sys 
#!{sys.prefix}/bin/pip install -q nbformat 
import nbformat 

In [ ]:
## run qBOLD toolbox
%run qBOLD_fun.ipynb

In [ ]:
## which contrast/baseline do we want to look at? ##
#############################################################
#baseline='rest'
baseline='control'
#conds=['calc']
#conds=['mem']

### calculate based on CBV-corrected calc (baseline CBV + median percchange CBV across 10 subjects) or without CBV-correction?
mode = 'corrected'
#mode = 'noCBV'
#############################################################

coords=(-30, -15, 0, 15, 30, 45, 60, 75, 90)

TE=30

for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    
    # download O2 sat & hematocrit values from xnat
    Hct = df_participants.query('participant_id==@sub')['Hct'].values
    O2sat = df_participants.query('participant_id==@sub')['O2sat'].values
    
    ## load R2` and %BOLD change
    R2s_baseline =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+baseline+'_space-T2_R2prime.nii')
    R2s_baseline_nii = nib.load(R2s_baseline)
    R2s_baseline_arr = np.array(R2s_baseline_nii.dataobj)
   # plotting.plot_img(R2s_baseline, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = -20, vmax=20, title=subj_id + ' R2s baseline')
    
    if ID < 56:
        conds=['mem', 'calc']
        #conds=['calc']
    if ID > 55:
        conds=['calc']
        
    for cond in conds:
        con = cond + baseline
        BOLD = os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-T2_BOLD_percchange.nii.gz') ## input: desc-preproc_bold.nii.gz from fmriprep
    
        #plotting.plot_img(BOLD, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = -2, vmax=2, title=sub + ' BOLD percchange')

        BOLD_nii = nib.load(BOLD)
        BOLD_arr = np.array(BOLD_nii.dataobj)

        

        ########################
        ## calculate R2'     ###
        ########################

        ##recalcalute task R2' based on R2' baseline and BOLD change
        R2s_Fujita = FujitaR2ptask_nii(R2s_baseline_arr, BOLD_arr, TE)
        ## turn into nifti and save
        R2s_Fujita_nii = nib.Nifti1Image(R2s_Fujita, R2s_baseline_nii.affine)
        nib.save(R2s_Fujita_nii, os.path.join(derivatives_dir, sub, 'qmri',   sub + '_task-'+cond+'_space-T2_desc-semi-quant_R2prime.nii.gz'))
        plotting.plot_img(R2s_Fujita_nii, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = -20, vmax=20, title=sub + ' R2s Fujita')
        plotting.plot_img(R2s_baseline, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = -20, vmax=20, title=sub + ' R2s baseline')

        ###################
        ## calculate OEF ##
        ###################
        ## download corrected CBV map
        if cond == 'calc': 
            if mode == 'corrected':## for calc, we take the corrected CBV values!
                CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-T2_cbv.nii')
            if mode == 'noCBV': ## if not corrected, we take baseline CBV values
                CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-T2_cbv.nii')
        if cond == 'mem': # for the other conditions, we take the CBV from the control task
            CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-T2_cbv.nii')
        CBV_nii = nib.load(CBV)
        CBV_arr = np.array(CBV_nii.dataobj)

        ##download brain mask
        BrMsk_CSF = os.path.join(derivatives_dir, sub, 'anat',  sub + '_BrMsk_CSF.nii')
        BrMsk_CSF_nii = nib.load(BrMsk_CSF)
        BrMsk_CSF_arr = np.array(BrMsk_CSF_nii.dataobj)
        #plotting.plot_img(BrMsk_CSF_nii, cmap='jet', threshold=0.5, cut_coords=coords, display_mode='z',colorbar = True, vmin = -1, vmax=1, title=subj_id + ' mask')

        mask = BrMsk_CSF_arr>0.5;

        # [Hz], physiologic constant derived from literature
        # 0.85: factor for small-vessel hematocrit
        C = 4/3*267.61918*np.pi*0.264*(Hct/100)*0.85*3; # 317 Hz

        rCBV = CBV_arr/100.0; #CBV fraction, e.g. CBV_WM = 0.015
        rCBV[rCBV < 0] = 0;
        eps=2.2204e-16 #to not divide by zero
        rOEF = (R2s_Fujita / (C*rCBV+eps)) * mask

        # reasonable boundary to avoid unreasonably high hot spots
        #rOEFmax_DSC = 5*np.nanmedian(rOEF[mask]);
        rOEFmax_DSC = 5*np.nanmedian(rOEF[mask]);
        if rOEFmax_DSC < 1.5:
                rOEFmax_DSC = 1.5 # FIXED Threshold
        rOEF[abs(rOEF) > rOEFmax_DSC] = rOEFmax_DSC
        rOEF = rOEF *mask

        rOEF_img = nib.Nifti1Image(rOEF, R2s_baseline_nii.affine)
        if (mode == 'corrected') and (cond =='calc'):
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant-corrected_oef.nii'))
        if (mode == 'noCBV') and (cond =='calc'):
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_oef.nii'))
        if (mode == 'noCBV') or (cond !='calc'):
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_oef.nii'))

        #####################
        ## calculate CMRO2 ##
        #####################

        # download CBF and OEF niftis from xnat
        CBF = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-T2_cbf.nii')
        #plotting.plot_img(CBF, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = -50, vmax=50, title=subj_id + ' CBF')

        CBF_nii = nib.load(CBF)
        CBF_np= np.array(CBF_nii.dataobj)
        # upscale CSF by 25% because of scanner
        CBF_upscaled = CBF_np / 0.75 # upscale CSF by 25% because of scanner
        
        if mode == 'corrected' and cond == 'calc':
            OEF_Fujita = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant-corrected_oef.nii')
        if mode == 'noCBV' and cond =='calc':
            OEF_Fujita = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_oef.nii')
        if mode == 'noCBV' or cond != 'calc':
            OEF_Fujita = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_oef.nii')

        OEF_nii = nib.load(OEF_Fujita)
        OEF_np= np.array(OEF_nii.dataobj)

        #calculate CaO2 value per subject, O2sat as a fraction
        CaO2 = 0.334 * Hct * 55.6 * float(O2sat)/100;

        ## calculate CMRO2 parameter map
        CMRO2 = CBF_upscaled * OEF_np * CaO2 /100;
        CMRO2 = CMRO2 * mask

        #store CMRO2 parameter map as nifti
        CMRO2_img = nib.Nifti1Image(CMRO2, CBV_nii.affine)
        if mode == 'corrected' and cond == 'calc':
            #nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant-corrected_cmro2.nii.gz'))
            nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant-corrected_cmro2.nii.gz'))            
        if mode == 'noCBV' and cond =='calc':
            #nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_cmro2.nii.gz'))                     
            nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_cmro2.nii.gz'))                     
        if mode == 'noCBV' or cond != 'calc':
            #nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_cmro2.nii.gz'))                     
            nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_cmro2.nii.gz'))                     
            
        ## plot output
        T1w_T2 = os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-T2_T1w.nii')
        plotting.plot_img(CMRO2_img, bg_img=T1w_T2, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=400, vmin = 10, threshold=0.1,
                   cut_coords=coords, title = 'CMRO2 semi-quant: '+ cond + ' ' + sub)
        if ID < 56:
            CMRO2_calc_img = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-T2_desc-orig_cmro2.nii') ## this is the original cmro2 image
        if ID > 55:
            CMRO2_calc_img = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-T2_desc-CBV_cmro2.nii') ## this is the original cmro2 image
            
        plotting.plot_img(CMRO2_calc_img, bg_img=T1w_T2, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=400, vmin = 10, threshold=0.1,
                   cut_coords=coords, title = 'CMRO2 orig: ' + cond + ' ' + sub)        

### semi-quantitative CMRO2, lowres space

In [ ]:
## which contrast/baseline do we want to look at? ##
#############################################################
#baseline='rest'
baseline='control'
#conds=['calc']
#conds=['mem']

### calculate based on CBV-corrected calc or without CBV-correction?
mode = 'corrected'
#mode = 'noCBV'
#############################################################

coords=(0, 15, 30, 45, 60, 75, 90)

TE=30

for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)
    
    # download O2 sat & hematocrit values from xnat
    Hct = df_participants.query('participant_id==@sub')['Hct'].values
    O2sat = df_participants.query('participant_id==@sub')['O2sat'].values
    
    ## load R2` and %BOLD change
    R2s_baseline =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+baseline+'_space-lowres_R2prime.nii.gz')
    R2s_baseline_nii = nib.load(R2s_baseline)
    R2s_baseline_arr = np.array(R2s_baseline_nii.dataobj)
    #plotting.plot_img(R2s_baseline, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, threshold=0, vmin =0, vmax=20, title=sub + ' R2s baseline')
    
    if ID < 56:
        conds=['mem', 'calc']
        conds=['calc']
    if ID > 55:
        conds=['calc']
        
    for cond in conds:
        con = cond + baseline
        BOLD = os.path.join(derivatives_dir, sub, 'func',  sub +  '_task-'+con+'_space-lowres_BOLD_percchange.nii.gz') ## input: desc-preproc_bold.nii.gz from fmriprep
    
        #plotting.plot_img(BOLD, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = -2, vmax=2, title=sub + ' BOLD percchange')

        BOLD_nii = nib.load(BOLD)
        BOLD_arr = np.array(BOLD_nii.dataobj)


        ########################
        ## calculate R2'     ###
        ########################

        ##recalcalute task R2' based on R2' baseline and BOLD change
        R2s_Fujita = FujitaR2ptask_nii(R2s_baseline_arr, BOLD_arr, TE)
        ## turn into nifti and save
        R2s_Fujita_nii = nib.Nifti1Image(R2s_Fujita, R2s_baseline_nii.affine)
        nib.save(R2s_Fujita_nii, os.path.join(derivatives_dir, sub, 'qmri',   sub + '_task-'+cond+'_space-lowres_desc-semi-quant_R2prime.nii.gz'))
        plotting.plot_img(R2s_Fujita_nii, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, threshold=0,vmin=0, vmax=15, title=sub + ' R2s Fujita')
        plotting.plot_img(R2s_baseline, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, threshold=0, vmin=0,vmax=15,title=sub + ' R2s baseline')

        ###################
        ## calculate OEF ##
        ###################
        ## download corrected CBV map
        if cond == 'calc': 
            if mode == 'corrected':## for calc, we take the corrected CBV values!
                CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-calc_space-lowres_cbv.nii.gz')
            if mode == 'noCBV': ## if not corrected, we take baseline CBV values
                CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-lowres_cbv.nii.gz')
        if cond == 'mem': # for the other conditions, we take the CBV from the control task
            CBV =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-control_space-lowres_cbv.nii.gz')
        CBV_nii = nib.load(CBV)
        CBV_arr = np.array(CBV_nii.dataobj)

        ##download brain mask: use lowres CBF image
        BrMsk_CSF = os.path.join(derivatives_dir, sub, 'qmri',   sub + '_task-control_space-lowres_cbf.nii.gz')
        BrMsk_CSF_nii = nib.load(BrMsk_CSF)
        BrMsk_CSF_arr = np.array(BrMsk_CSF_nii.dataobj)
        #plotting.plot_img(BrMsk_CSF_nii, cmap='jet', threshold=3, cut_coords=coords, display_mode='z',colorbar = True, vmin = -1, vmax=1, title=sub + ' mask')

        mask = BrMsk_CSF_arr>3;

        # [Hz], physiologic constant derived from literature
        # 0.85: factor for small-vessel hematocrit
        C = 4/3*267.61918*np.pi*0.264*(Hct/100)*0.85*3; # 317 Hz

        rCBV = CBV_arr/100.0; #CBV fraction, e.g. CBV_WM = 0.015
        rCBV[rCBV < 0] = 0;
        eps=2.2204e-16 #to not divide by zero
        rOEF = (R2s_Fujita / (C*rCBV+eps)) * mask

        # reasonable boundary to avoid unreasonably high hot spots
        #rOEFmax_DSC = 5*np.nanmedian(rOEF[mask]);
        rOEFmax_DSC = 5*np.nanmedian(rOEF[mask]);
        if rOEFmax_DSC < 1.5:
                rOEFmax_DSC = 1.5 # FIXED Threshold
        rOEF[abs(rOEF) > rOEFmax_DSC] = rOEFmax_DSC
        rOEF = rOEF *mask

        rOEF_img = nib.Nifti1Image(rOEF, R2s_baseline_nii.affine)
        if (mode == 'corrected') and (cond =='calc'):
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant-corrected_oef.nii.gz'))
        if (mode == 'noCBV') and (cond =='calc'):
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant_oef.nii.gz'))
        if (mode == 'noCBV') or (cond !='calc'):
            nib.save(rOEF_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant_oef.nii.gz'))
        plotting.plot_img(rOEF_img, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = 0, vmax=1, title=sub + ' OEF semi-quant')
        OEF_orig = os.path.join(derivatives_dir, sub, 'qmri',   sub + '_task-control_space-lowres_oef.nii.gz')
        plotting.plot_img(OEF_orig, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, vmin = 0, vmax=1, title=sub + ' OEF orig control')

        #####################
        ## calculate CMRO2 ##
        #####################

        # download CBF and OEF niftis from xnat
        CBF = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_cbf.nii.gz')
        plotting.plot_img(CBF, cmap='jet', cut_coords=coords, display_mode='z',colorbar = True, threshold=0, vmax=50, title=sub + ' CBF')

        CBF_nii = nib.load(CBF)
        CBF_np= np.array(CBF_nii.dataobj)
        # upscale CSF by 25% because of scanner
        CBF_upscaled = CBF_np / 0.75 # upscale CSF by 25% because of scanner
        
        if mode == 'corrected' and cond == 'calc':
            OEF_Fujita = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant-corrected_oef.nii.gz')
        if mode == 'noCBV' and cond =='calc':
            OEF_Fujita = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant_oef.nii.gz')
        if mode == 'noCBV' or cond != 'calc':
            OEF_Fujita = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant_oef.nii.gz')

        OEF_nii = nib.load(OEF_Fujita)
        OEF_np= np.array(OEF_nii.dataobj)

        #calculate CaO2 value per subject, O2sat as a fraction
        CaO2 = 0.334 * Hct * 55.6 * float(O2sat)/100;

        ## calculate CMRO2 parameter map
        CMRO2 = CBF_upscaled * OEF_np * CaO2 /100;
        CMRO2 = CMRO2 * mask

        #store CMRO2 parameter map as nifti
        CMRO2_img = nib.Nifti1Image(CMRO2, CBV_nii.affine)
        if mode == 'corrected' and cond == 'calc':
            nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant-corrected_cmro2.nii.gz'))            
        if mode == 'noCBV' and cond =='calc':
            nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant_cmro2.nii.gz'))                     
        if mode == 'noCBV' or cond != 'calc':
            nib.save(CMRO2_img, os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-lowres_desc-semi-quant_cmro2.nii.gz'))                     
            
        ## plot output
        #T1w_lowres = os.path.join(derivatives_dir, sub, 'anat',  sub + '_space-lowres_T1w_brain.nii.gz')
        plotting.plot_img(CMRO2_img,cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=400, vmin = 10, threshold=0.1,
                   cut_coords=coords, title = 'CMRO2 semi-quant: '+ cond + ' ' + sub)
        if mode == 'corrected' and cond == 'calc':
            CMRO2_semqu_highres =  os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant-corrected_cmro2.nii.gz') ## this is the original cmro2 image
            CMRO2_quant = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_desc-CBV_cmro2.nii.gz') ## this is the original cmro2 image
        if mode == 'noCBV' or cond != 'calc':
            CMRO2_semqu_highres = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_base-'+baseline+'_space-T2_desc-semi-quant_cmro2.nii.gz') ## this is the original cmro2 image
            CMRO2_quant = os.path.join(derivatives_dir, sub, 'qmri',  sub + '_task-'+cond+'_space-lowres_desc-orig_cmro2.nii.gz') ## this is the original cmro2 image
            
        plotting.plot_img(CMRO2_semqu_highres, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=400, vmin = 10, threshold=0.1,
                   cut_coords=coords, title = 'CMRO2 semi-quant T2: ' + cond + ' ' + sub)   
        plotting.plot_img(CMRO2_quant, cmap='jet',display_mode='z', colorbar=True, black_bg=True, vmax=400, vmin = 10, threshold=0.1,
                   cut_coords=coords, title = 'CMRO2 quant lowres: ' + cond + ' ' + sub)   


### normalize parameter maps

In [ ]:
%%bash
. /etc/fsl/5.0/fsl.sh
. /etc/ants/ants.sh

parameters="cmro2 R2prime T2Smap oef cbv"
#parameters="cmro2 cbf oef"
#parameters="T2Smap"

sids="19 20 21 23 26 27 28 29 30 31 32 34 33 35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55 58 59 60 61 63 64 65 66 67 68"
#sids="19 20 21 23 26 27 28 29 30 31 32 34 33 35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55"
#sids="35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55 58 59 60 61 63 64 65 66 67 68"


for ID in $sids; 
do
    ##define directories
    SID=p0${ID}
    echo $SID
    base_path="/root_dir/" 
    locfolder="${base_path}/data/derivatives/sub-${SID}"
    
    ##define input
    ## output from fmriprep
    T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w.nii.gz"
    br_mask="${locfolder}/anat/sub-${SID}_desc-fmriprep_brain_mask.nii.gz"
    transform_to_standard="${locfolder}/anat/sub-${SID}_from-T1w_to-MNI152NLin6Asym_mode-image_xfm.h5"
    #echo $transform_to_standard
    T1w_MNIspace="${locfolder}/anat/sub-${SID}_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz"
    #echo $T1w_MNIspace
    ## output from qmri
    T1w_T2space="${locfolder}/qmri/sub-${SID}_space-T2_T1w.nii"
    
    ##define output
    T1w_brain="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w_brain.nii.gz"
    T2_to_T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_T2_to_T1w.mat"
    
    ##brain-extract T1w
    echo "apply brain mask for subject ${SID} (BET)"
    #fslmaths $T1w -mas $br_mask $T1w_brain
    #bet $T1w $T1w_brain
    
    ##get T2 to T1w transformation mat
    #flirt -in $T1w_T2space -ref $T1w_brain -omat $T2_to_T1w -dof 6

    
    for par in $parameters; do    
        if [ "$ID" -lt "56" ]; then
        if [ par != "cbv" ]; then
            conditions="rest control mem calc"
            #conditions="mem"
        fi
        fi
        if [ "$ID" -gt "55" ]; then
            conditions="control calc"
        fi
        if [ $par == "cbv" ]; then
        if [ "$ID" -lt "56" ]; then
            conditions="control"
        fi
        fi
      
        
        for cond in $conditions; do
            echo $cond
        
            if [ $par != "cmro2" ]; then
                parameter_map="${locfolder}/qmri/sub-${SID}_task-${cond}_space-T2_${par}.nii"
                ##define outputs
                out_T1w="${locfolder}/qmri/sub-${SID}_task-${cond}_space-T1w_${par}.nii.gz"
                out_MNI="${locfolder}/qmri/sub-${SID}_task-${cond}_space-MNI152_${par}.nii.gz"
            fi
            
            if [ $par == "cmro2" ] && [ $cond != "calc" ]; then
                parameter_map="${locfolder}/qmri/sub-${SID}_task-${cond}_space-T2_desc-orig_${par}.nii"
                out_T1w="${locfolder}/qmri/sub-${SID}_task-${cond}_T1w-space_desc-orig_${par}.nii.gz"
                out_MNI="${locfolder}/qmri/sub-${SID}_task-${cond}_space-MNI152_desc-orig_${par}.nii.gz"
            fi
            if [ $par == "cmro2" ] && [ $cond == "calc" ] && [ $ID < 56 ]; then ##non-CBV corrected, but only for ID<56
                parameter_map="${locfolder}/qmri/sub-${SID}_task-${cond}_space-T2_desc-orig_${par}.nii"
                out_T1w="${locfolder}/qmri/sub-${SID}_task-${cond}_T1w-space_desc-orig_${par}.nii.gz"
                out_MNI="${locfolder}/qmri/sub-${SID}_task-${cond}_space-MNI152_desc-orig_${par}.nii.gz"
            fi
            
            ##apply transformation from lowres T2 to highres T1 space for parameter maps
            flirt -in $parameter_map -ref $T1w_brain -applyxfm -init $T2_to_T1w -out $out_T1w

            ##apply transformation (saved by fmriprep) from T1 to standard space
            antsApplyTransforms -d 3 --float 1 --verbose 1 -i $out_T1w -r $T1w_MNIspace -t $transform_to_standard -o $out_MNI
            
            if [ $par == "cmro2" ] && [ $cond == "calc" ]; then ## all CBV-corrected
                parameter_map="${locfolder}/qmri/sub-${SID}_task-${cond}_space-T2_desc-CBV_${par}.nii" 
                out_T1w="${locfolder}/qmri/sub-${SID}_task-${cond}_T1w-space_desc-CBV_${par}.nii.gz"
                out_MNI="${locfolder}/qmri/sub-${SID}_task-${cond}_space-MNI152_desc-CBV_${par}.nii.gz"
            
                ##apply transformation from lowres T2 to highres T1 space for parameter maps
                flirt -in $parameter_map -ref $T1w_brain -applyxfm -init $T2_to_T1w -out $out_T1w

                ##apply transformation (saved by fmriprep) from T1 to standard space
                antsApplyTransforms -d 3 --float 1 --verbose 1 -i $out_T1w -r $T1w_MNIspace -t $transform_to_standard -o $out_MNI
            fi

            
        done
    done
done


In [ ]:
## show normalized parameter maps
parameters = ['cbf', 'cmro2', 'oef']

for par in parameters:
    for ID in sids:
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)
        sub_ID = "p{:03d}".format(ID) 

        if ID < 56 and par !='cbv':
            conditions=['rest', 'control', 'mem', 'calc']

        if ID > 55: 
            conditions=['control', 'calc']

        if par == 'cbv' and ID < 56:
            conditions=['control']

        
        for cond in conditions: 
            if par != 'cmro2':
                par_MNI = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_space-MNI152_' + par + '.nii.gz')
            if par == 'cmro2' and cond != 'calc':
                par_MNI = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_space-MNI152_desc-orig_' + par + '.nii.gz')
            if par == 'cmro2' and cond == 'calc':
                par_MNI = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_space-MNI152_desc-orig_' + par + '.nii.gz')

            T1w_MNIspace = os.path.join(derivatives_dir, sub, 'anat',sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz')
            plotting.plot_img(par_MNI,  bg_img =T1w_MNIspace, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0.5, black_bg=True, title = par + ' : ' + sub)


### normalize T2map

In [ ]:
%%bash
. /etc/fsl/5.0/fsl.sh
. /etc/ants/ants.sh

##only T2 map
parameters="T2map"

sids="19 20 21 23 26 27 28 29 30 31 32 34 33 35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55 58 59 60 61 63 64 65 66 67 68"
#sids="58 59 60 61 63 64 65 66 67 68 "
#sids="20"

## T2map, no condition
for ID in $sids; 
do
    ##define directories
    SID=p0${ID}
    echo $SID
    base_path="/root_dir/" 
    locfolder="${base_path}/data/derivatives/sub-${SID}"
    
    ##define input
    ## output from fmriprep
    T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w.nii.gz"
    br_mask="${locfolder}/anat/sub-${SID}_desc-fmriprep_brain_mask.nii.gz"
    transform_to_standard="${locfolder}/anat/sub-${SID}_from-T1w_to-MNI152NLin6Asym_mode-image_xfm.h5"
    #echo $transform_to_standard
    T1w_MNIspace="${locfolder}/anat/sub-${SID}_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz"
    #echo $T1w_MNIspace
    ## output from qmri
    T1w_T2space="${locfolder}/qmri/sub-${SID}_space-T2_T1w.nii"
    
    ##define output
    T1w_brain="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w_brain.nii.gz"
    T2_to_T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_T2_to_T1w.mat"
    
    ##brain-extract T1w
    echo "apply brain mask for subject ${SID} (BET)"
    fslmaths $T1w -mas $br_mask $T1w_brain
    #bet $T1w $T1w_brain
    
    ##get T2 to T1w transformation mat
    flirt -in $T1w_T2space -ref $T1w_brain -omat $T2_to_T1w -dof 6

    
    for par in $parameters;
    do    

        
        parameter_map="${locfolder}/qmri/sub-${SID}_space-T2_T2map.nii"


        ##define outputs
        out_T1w="${locfolder}/qmri/sub-${SID}_space-T1w_T2map.nii.gz"
        out_MNI="${locfolder}/qmri/sub-${SID}_space-MNI152_T2map.nii.gz"



           ##apply transformation from lowres T2 to highres T1 space for parameter maps
        flirt -in $parameter_map -ref $T1w_brain -applyxfm -init $T2_to_T1w -out $out_T1w

        ##apply transformation (saved by fmriprep) from T1 to standard space
        antsApplyTransforms -d 3 --float 1 --verbose 1 -i $out_T1w -r $T1w_MNIspace -t $transform_to_standard -o $out_MNI

    done
done

## normalize semi-quant CMRO2 maps

In [ ]:
%%bash
. /etc/fsl/5.0/fsl.sh
. /etc/ants/ants.sh

parameters="oef cmro2"
parameters="cmro2"

sids="19 20 21 23 26 27 28 29 30 31 32 34 33 35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55 58 59 60 61 63 64 65 66 67 68"
#sids="19 20 21 23 26 27 28 29 30 31 32 34 33 35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55"

#sids="19"

for par in $parameters; do
for ID in $sids; 
do
    ##define directories
    SID=p0${ID}
    echo $SID
    base_path="/root_dir/" 
    locfolder="${base_path}/data/derivatives/sub-${SID}"
    
    ##define input
    ## output from fmriprep
    T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w.nii.gz"
    br_mask="${locfolder}/anat/sub-${SID}_desc-fmriprep_brain_mask.nii.gz"
    transform_to_standard="${locfolder}/anat/sub-${SID}_from-T1w_to-MNI152NLin6Asym_mode-image_xfm.h5"
    #echo $transform_to_standard
    T1w_MNIspace="${locfolder}/anat/sub-${SID}_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz"
    #echo $T1w_MNIspace
    ## output from qmri
    T1w_T2space="${locfolder}/qmri/sub-${SID}_space-T2_T1w.nii"
    
    ##define output
    T1w_brain="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w_brain.nii.gz"
    T2_to_T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_T2_to_T1w.mat"
    
    ##brain-extract T1w
    #echo "apply brain mask for subject ${SID} (BET)"
    #fslmaths $T1w -mas $br_mask $T1w_brain
    #bet $T1w $T1w_brain
    
    ##get T2 to T1w transformation mat
    #flirt -in $T1w_T2space -ref $T1w_brain -omat $T2_to_T1w -dof 6

    
    if [ "$ID" -lt "56" ]; then
        if [ par != "cbv" ]; then
            #tasks="calc mem"
            tasks="calc"
            baseline="control"
            #baseline="rest"
        fi
    fi
    if [ "$ID" -gt "55" ]; then
            tasks="calc"
            baseline="control"
    fi

        
    for task in $tasks; do            
                
        parameter_map="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T2_desc-semi-quant_${par}.nii.gz"
        out_T1w="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T1w_desc-semi-quant_${par}.nii.gz"
        out_MNI="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-MNI152_desc-semi-quant_${par}.nii.gz"

        if [ $task == "calc" ]; then
                parameter_map="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T2_desc-semi-quant_${par}.nii.gz"
                out_T1w="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T1w_desc-semi-quant_${par}.nii.gz"
                out_MNI="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-MNI152_desc-semi-quant_${par}.nii.gz"
        fi
            
            ##apply transformation from lowres T2 to highres T1 space for parameter maps
        flirt -in $parameter_map -ref $T1w_brain -applyxfm -init $T2_to_T1w -out $out_T1w

            ##apply transformation (saved by fmriprep) from T1 to standard space
        antsApplyTransforms -d 3 --float 1 --verbose 1 -i $out_T1w -r $T1w_MNIspace -t $transform_to_standard -o $out_MNI

        ## also normalize CBV-corrected cmro2 maps
        if [ $task == "calc" ]; then
            parameter_map="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T2_desc-semi-quant-corrected_${par}.nii.gz"
            out_T1w="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T1w_desc-semi-quant-corrected_${par}.nii.gz"
            out_MNI="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-MNI152_desc-semi-quant-corrected_${par}.nii.gz"
            ##apply transformation from lowres T2 to highres T1 space for parameter maps
            flirt -in $parameter_map -ref $T1w_brain -applyxfm -init $T2_to_T1w -out $out_T1w

            ##apply transformation (saved by fmriprep) from T1 to standard space
            antsApplyTransforms -d 3 --float 1 --verbose 1 -i $out_T1w -r $T1w_MNIspace -t $transform_to_standard -o $out_MNI
        fi
            

        
    done
done
done


In [ ]:
## show normalized parameter maps
parameters = ['cmro2', 'oef']

for par in parameters:
    for ID in sids:
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)
        sub_ID = "p{:03d}".format(ID) 

        if ID < 56 and par !='cbv':
            tasks=['mem', 'calc']
            baseline='control'

        if ID > 55: 
            tasks=[ 'calc']
            baseline='control'



        for cond in tasks:
            par_MNI = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_base-'+baseline+'_space-MNI152_desc-semi-quant_' + par + '.nii.gz')
            
            T1w_MNIspace = os.path.join(derivatives_dir, sub, 'anat',sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz')
            plotting.plot_img(par_MNI,  bg_img =T1w_MNIspace, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, black_bg=True, title = par + ' ' + sub)
        
        
            if cond == 'calc':
                par_MNI = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_base-'+baseline+'_space-MNI152_desc-semi-quant-corrected_' + par + '.nii.gz')

                T1w_MNIspace = os.path.join(derivatives_dir, sub, 'anat',sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz')
                plotting.plot_img(par_MNI,  bg_img =T1w_MNIspace, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, black_bg=True, title = par + ' corrected ' + sub)


In [ ]:
#resolution='lowres'
resolution='res-2'
#CMRO2_quant = 'fully-quant'
CMRO2_quant = 'semi-quant'
MNI_space='yes'

task='calc'
baseline='control'
CMRO2_mode='corrected'

PLS_BOLD = results_dir + '/N40_BSR_meanPLS_func_clusterBSR_calccontrol_GMmask.nii.gz'
PLS_BOLD_mask = results_dir + '/N40_BSR_meanPLS_func_clusterBSR_calccontrol_GMmask_bin.nii.gz'
    
## create mask that includes both posiitve and negative BOLD areas
! fslmaths {PLS_BOLD} -abs -bin {PLS_BOLD_mask}
masker = input_data.NiftiMasker(mask_img = PLS_BOLD_mask)
mask_arr = np.array(nib.load(PLS_BOLD_mask).dataobj)
mask_len = len (mask_arr[abs(mask_arr)>0])
    

BOLD_percchange = np.zeros((len(sids), mask_len))
CBF_percchange = np.zeros((len(sids), mask_len))
CMRO2_percchange = np.zeros((len(sids), mask_len))


ID = 44
i = 0
sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
print(sub)

sub_dir = os.path.join(data_dir, sub)

dir_anat = os.path.join(sub_dir, 'anat')
dir_func = os.path.join(sub_dir, 'func')
dir_perf = os.path.join(sub_dir, 'perf')

dir_anat_deriv = os.path.join(derivatives_dir, sub, 'anat')
dir_func_deriv = os.path.join(derivatives_dir, sub, 'func')
dir_perf_deriv = os.path.join(derivatives_dir, sub, 'perf')
dir_qmri_deriv = os.path.join(derivatives_dir, sub, 'qmri')

if MNI_space=='yes':
    #load percent signal change image
    if resolution != 'lowres':
        BOLD_percchange_nii = nib.load(dir_func_deriv + '/' + sub + '_task-'+task + baseline+'_space-MNI152_desc-fmriprep_BOLD_percchange.nii.gz')
        BOLD_percchange[i, :] = masker.fit_transform(BOLD_percchange_nii) 
    if resolution == 'lowres':
        BOLD_percchange_nii = nib.load(dir_func_deriv + '/' + sub + '_task-'+task + baseline+'_space-MNI152-lowres_desc-fmriprep_BOLD_percchange.nii.gz')
        BOLD_percchange[i, :] = masker.fit_transform(BOLD_percchange_nii) 

    ## for CBF, create %change    
    if resolution != 'lowres':
        CBF_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_cbf.nii.gz'))/0.75
        CBF_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_cbf.nii.gz'))/0.75
    if resolution == 'lowres':
        CBF_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_cbf.nii.gz'))/0.75
        CBF_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_cbf.nii.gz'))/0.75        
    CBF_percchange[i, :]  = (CBF_task - CBF_baseline) / CBF_baseline *100


    ## for CMR02, create %change
    ## baseline
    if resolution != 'lowres':
        CMRO2_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_desc-orig_cmro2.nii.gz'))
    if resolution == 'lowres':
        CMRO2_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_desc-orig_cmro2.nii.gz'))

    if CMRO2_quant == 'fully-quant':
        ## task
        if CMRO2_mode == 'corrected' and task == 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_desc-CBV_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_desc-CBV_cmro2.nii.gz'))
        if CMRO2_mode == 'orig' or task != 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_desc-orig_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_desc-orig_cmro2.nii.gz'))

    if CMRO2_quant == 'semi-quant':
        ## task
        if CMRO2_mode == 'orig' or task !='calc': ## without CBV correction
            if resolution != 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152_desc-semi-quant_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant_cmro2.nii.gz'))
        if CMRO2_mode == 'corrected' and task == 'calc':  
            if resolution != 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152_desc-semi-quant-corrected_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant-corrected_cmro2.nii.gz'))

    CMRO2_percchange[i, :]  = (CMRO2_task- CMRO2_baseline) / CMRO2_baseline *100 
    CMRO2_percchange[i, :][np.isinf(CMRO2_percchange[i, :])]  = np.nan

if MNI_space=='no':
    if resolution != 'lowres':
        BOLD_percchange_nii = nib.load(dir_func_deriv + '/' + sub + '_task-'+task + baseline+'_space-T2_BOLD_percchange.nii.gz')
        BOLD_percchange[i, :] = masker.fit_transform(BOLD_percchange_nii) 
    if resolution == 'lowres':
        BOLD_percchange_nii = nib.load(dir_func_deriv + '/' + sub + '_task-'+task + baseline+'_space-lowres_BOLD_percchange.nii.gz')
        BOLD_percchange[i, :] = masker.fit_transform(BOLD_percchange_nii) 

    ## for CBF, create %change    
    if resolution != 'lowres':
        CBF_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-T2_cbf.nii'))/0.75
        CBF_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-T2_cbf.nii'))/0.75
    if resolution == 'lowres':
        CBF_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-lowres_cbf.nii.gz'))/0.75
        CBF_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-lowres_cbf.nii.gz'))/0.75        
    CBF_percchange[i, :]  = (CBF_task - CBF_baseline) / CBF_baseline *100


    ## for CMR02, create %change
    ## baseline
    if resolution != 'lowres':
        CMRO2_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-T2_desc-orig_cmro2.nii'))
    if resolution == 'lowres':
        CMRO2_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-lowres_desc-orig_cmro2.nii.gz'))

    if CMRO2_quant == 'fully-quant':
        ## task
        if CMRO2_mode == 'corrected' and task == 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-T2_desc-CBV_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-lowres_desc-CBV_cmro2.nii.gz'))
        if CMRO2_mode == 'orig' or task != 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-T2_desc-orig_cmro2.nii'))
            if resolution == 'lowres':
                CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-lowres_desc-orig_cmro2.nii.gz'))

    if CMRO2_quant == 'semi-quant':
        ## task
        if CMRO2_mode == 'orig' or task !='calc': ## without CBV correction
            if resolution != 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-T2_desc-semi-quant_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-lowres_desc-semi-quant_cmro2.nii.gz'))
        if CMRO2_mode == 'corrected' and task == 'calc':  
            if resolution != 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-T2_desc-semi-quant-corrected_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-lowres_desc-semi-quant-corrected_cmro2.nii.gz'))

    CMRO2_percchange[i, :]  = (CMRO2_task- CMRO2_baseline) / CMRO2_baseline *100 
    CMRO2_percchange[i, :][np.isinf(CMRO2_percchange[i, :])]  = np.nan

### normalize lowres semi-quant maps

In [ ]:
%%bash
. /etc/fsl/5.0/fsl.sh
. /etc/ants/ants.sh

parameters="cmro2, oef"

sids="19 20 21 23 26 27 28 29 30 31 32 34 33 35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55 58 59 60 61 63 64 65 66 67 68"
#sids="19 20 21 23 26 27 28 29 30 31 32 34 33 35 36 37 38 39 40 43 44 46 47 48 49 50 51 52 54 55"

for par in $parameters; do
for ID in $sids; 
do
    ##define directories
    SID=p0${ID}
    echo $SID
    base_path="/home/tumnic/sepp/neuroenergeticslab_notebooks/erc-wp1/manuscript" 
    locfolder="${base_path}/data/derivatives/sub-${SID}"
    
    ##define input
    ## output from fmriprep
    T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w.nii.gz"
    br_mask="${locfolder}/anat/sub-${SID}_desc-fmriprep_brain_mask.nii.gz"
    transform_to_standard="${locfolder}/anat/sub-${SID}_from-T1w_to-MNI152NLin6Asym_mode-image_xfm.h5"
    #echo $transform_to_standard
    T1w_MNIspace="${locfolder}/anat/sub-${SID}_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz"
    #echo $T1w_MNIspace
    ## output from qmri
    T1w_lowresspace="${locfolder}/qmri/sub-${SID}_space-lowres_T1w_brain.nii.gz"
    
    ##define output
    T1w_brain="${locfolder}/anat/sub-${SID}_desc-fmriprep_T1w_brain.nii.gz"
    lowres_to_T1w="${locfolder}/anat/sub-${SID}_desc-fmriprep_lowres_to_T1w.mat"
    
    ##brain-extract T1w
    #echo "apply brain mask for subject ${SID} (BET)"
    #fslmaths $T1w -mas $br_mask $T1w_brain
    #bet $T1w $T1w_brain
    
    ##get lowres to T1w transformation mat
    flirt -in $T1w_lowresspace -ref $T1w_brain -omat $lowres_to_T1w -dof 6

    
    if [ "$ID" -lt "56" ]; then
        if [ par != "cbv" ]; then
            #tasks="calc mem"
            tasks="calc"
            baseline="control"
            #baseline="rest"
        fi
    fi
    if [ "$ID" -gt "55" ]; then
            tasks="calc"
            baseline="control"
    fi

        
    for task in $tasks; do            
                
        parameter_map="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-lowres_desc-semi-quant_${par}.nii.gz"
        out_T1w="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T1w_desc-semi-quant_${par}.nii.gz"
        out_MNI="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-MNI152-lowres_desc-semi-quant_${par}.nii.gz"

        ## also normalize CBV-corrected  maps
        if [ $task == "calc" ]; then
                parameter_map="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-lowres_desc-semi-quant-corrected_${par}.nii.gz"
                out_T1w="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-T1w_desc-semi-quant-corrected_${par}.nii.gz"
                out_MNI="${locfolder}/qmri/sub-${SID}_task-${task}_base-${baseline}_space-MNI152-lowres_desc-semi-quant-corrected_${par}.nii.gz"
        fi
            
            ##apply transformation from lowres lowres to highres T1 space for parameter maps
        flirt -in $parameter_map -ref $T1w_brain -applyxfm -init $lowres_to_T1w -out $out_T1w

            ##apply transformation (saved by fmriprep) from T1 to standard space
        antsApplyTransforms -d 3 --float 1 --verbose 1 -i $out_T1w -r $T1w_MNIspace -t $transform_to_standard -o $out_MNI
            
        
    done
done
done


In [ ]:
## show normalized parameter maps
parameters = ['cmro2', 'oef']

for par in parameters:
    for ID in sids:
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)
        sub_ID = "p{:03d}".format(ID) 

        if ID < 56 and par !='cbv':
            tasks=['mem', 'calc']
            baseline='control'

        if ID > 55: 
            tasks=[ 'calc']
            baseline='control'



        for cond in tasks:
            MNI_lowres = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant_' + par + '.nii.gz')
            MNI_highres = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_base-'+baseline+'_space-MNI152_desc-semi-quant_' + par + '.nii.gz')
            
            T1w_MNIspace = os.path.join(derivatives_dir, sub, 'anat',sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz')
            plotting.plot_img(MNI_lowres,  bg_img =T1w_MNIspace, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, vmax=400, black_bg=True, title = par + ' ' + sub+ ' ' + cond + ' lowres')
            plotting.plot_img(MNI_highres,  bg_img =T1w_MNIspace, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, vmax=400, black_bg=True, title = par + ' ' + sub + ' ' + cond+ ' highres')
        
        
            if cond == 'calc':
                MNI_lowres = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant-corrected_' + par + '.nii.gz')
                MNI_highres = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ cond +'_base-'+baseline+'_space-MNI152_desc-semi-quant-corrected_' + par + '.nii.gz')

                T1w_MNIspace = os.path.join(derivatives_dir, sub, 'anat',sub + '_space-MNI152NLin6Asym_res-2_desc-preproc_T1w.nii.gz')
                plotting.plot_img(MNI_lowres,  bg_img =T1w_MNIspace, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0,vmax=400,  black_bg=True, title = par + ' corrected ' + sub+  ' ' + cond+' lowres')
                plotting.plot_img(MNI_highres,  bg_img =T1w_MNIspace, cmap='jet',display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0,vmax=400,  black_bg=True, title = par + ' corrected ' + sub+ ' ' + cond+ ' highres')


## create MNI-space CBV + R2' mask

In [ ]:
## load all subject's CBV baseline maps, take median across subjects, plot distribution and apply thresholds
sids = [58, 59, 60, 61, 63, 64, 65, 66, 67, 68]

YEO_SNR_mask = os.path.join( derivatives_dir, 'task-all_space-MNI152_res-2_SNR_YEO_group_mask.nii.gz')
plotting.plot_img(YEO_SNR_mask,cmap = 'jet_r', display_mode='z', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, black_bg=True, title = 'SNR Yeo mask')

baseline='control'
masker = input_data.NiftiMasker(mask_img=YEO_SNR_mask)

mask = np.array(nib.load(YEO_SNR_mask).dataobj)

CBV_subj = np.zeros((len(sids), len(np.argwhere(mask))))
R2s_subj = np.zeros((len(sids), len(np.argwhere(mask))))

for i, ID in enumerate(sids):
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    par_CBV = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ baseline +'_space-MNI152_cbv.nii.gz')
    par_R2s= os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ baseline +'_space-MNI152_R2prime.nii.gz')
    
    CBV_subj[i, :] = masker.fit_transform(par_CBV)
    R2s_subj[i, :] = masker.fit_transform(par_R2s)
            
median_CBV_subj = np.nanmedian(CBV_subj, axis=0)
median_R2s_subj = np.nanmedian(R2s_subj, axis=0)

fig, ax = plt.subplots(1)
plt.hist(median_CBV_subj, bins=50)
plt.show()
fig, ax = plt.subplots(1)
plt.hist(median_R2s_subj, bins=50)
plt.show()

## create mask:
## make niftis out of median CBV and R2'
CBV_nii = masker.inverse_transform(median_CBV_subj)
nib.save(CBV_nii, os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_CBV_subj_median.nii.gz'))
CBV_nifti = os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_CBV_subj_median.nii.gz')
R2s_nii = masker.inverse_transform(median_R2s_subj)
nib.save(R2s_nii, os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_R2s_subj_median.nii.gz'))
R2s_nifti =  os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_R2s_subj_median.nii.gz')

## compute thresholds
CBV_median = np.nanmedian(median_CBV_subj)
absolute_dev_CBV = np.nanmedian(abs(median_CBV_subj-CBV_median))
thr_CBV = CBV_median + 2.5*absolute_dev_CBV
print('CBV threshold: ' + str(thr_CBV))

R2s_median = np.nanmedian(median_R2s_subj)
absolute_dev_R2s = np.nanmedian(abs(median_R2s_subj-R2s_median))
thr_R2s = R2s_median + 2.5*absolute_dev_R2s
print('R2s threshold: ' + str(thr_R2s))

## apply thresholds
CBV_mask = os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_CBV_mask.nii.gz')
! fslmaths {CBV_nifti} -uthr {thr_CBV} -bin {CBV_mask}
plotting.plot_img(CBV_mask,display_mode='z', cmap='jet_r', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, black_bg=True, title = 'CBV mask')
R2s_mask = os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_R2s_mask.nii.gz')
! fslmaths {R2s_nifti} -uthr {thr_R2s} -bin {R2s_mask}
plotting.plot_img(R2s_mask,display_mode='z', cmap='jet_r', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, black_bg=True, title = 'R2s mask')

## merge masks
CBV_R2s_mask = os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_CBV_R2s_mask.nii.gz')
! fslmaths {CBV_mask} -mas {R2s_mask} -bin {CBV_R2s_mask}

plotting.plot_img(CBV_R2s_mask,display_mode='z', cmap='jet_r', cut_coords = (0, 10, 20, 30, 40, 50), threshold = 0, black_bg=True, title = 'CBV R2s mask')


## get median of CBV before and after masking
median_CBV_nomask = np.nanmedian(median_CBV_subj[median_CBV_subj>0])
print(str(median_CBV_nomask))

masker_CBV_R2s_masker = input_data.NiftiMasker(mask_img=CBV_R2s_mask)
median_CBV_masked = np.nanmedian(masker_CBV_R2s_masker.fit_transform(CBV_nii))
print(str(median_CBV_masked))


##boxplots: subj GM median before and after masking ##
#####################
subj_median_CBV_nomask = np.nanmedian(CBV_subj, axis=1)

mask_len = np.array(nib.load(CBV_R2s_mask).dataobj)
CBV_subj_masked = np.zeros((len(sids), len(np.argwhere(mask_len))))

for i, ID in enumerate(sids):
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    par_CBV = os.path.join(derivatives_dir, sub, 'qmri',sub + '_task-'+ baseline +'_space-MNI152_cbv.nii.gz')
    
    CBV_subj_masked[i, :] = masker_CBV_R2s_masker.fit_transform(par_CBV)
    
subj_median_CBV_masked = np.nanmedian(CBV_subj_masked, axis=1)

fig, ax = plt.subplots(1)
sns.boxplot([subj_median_CBV_nomask, subj_median_CBV_masked])
sns.stripplot([subj_median_CBV_nomask, subj_median_CBV_masked])
ax.set_ylabel('CBV')
ax.set_xticklabels(['GM masked', 'GM + CBV + R2s masked'])

## threshold native-space parameter maps

In [ ]:
## run qBOLD toolbox
%run qBOLD_fun.ipynb

In [ ]:
#grey matter mask
GMthresh = 0.5; #mandatory

#mask CSF influenced areas
T2thresh = 90; #mandatory

#mask R2strich artifact areas
R2thresh = 9 ; #mandatory

#mask CBV 
CBVthresh = 10; #optional: comment out if not used, default = 20

#mask T2S
T2Sthresh = 90; #optional: comment out if not used, default = 150

#mask OEF
OEFthresh = 0.9; #optional: comment out if not used, default = 1

#mask CBF
CBFthresh = 90; #optional: comment out if not used, default = 150

In [ ]:
parameters=['T2map', 'T2Smap', 'cbv', 'cbf', 'R2prime', 'oef', 'cmro2']
#parameters=['func', 'cbf', 'cmro2']

for p, par in enumerate(parameters):
    for i, ID in enumerate(sids): #loop over subjects
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)

        dir_deriv = os.path.join(data_dir, 'derivatives', sub)
    


        if ID < 56:
            conds = ['rest', 'control', 'mem', 'calc']
        if ID > 55:
            conds = ['control', 'calc']   

        mask_list = create_qBOLD_masks(sub, dir_deriv, conds, GMthresh, T2thresh, R2thresh, CBVthresh, T2Sthresh, OEFthresh,  CBFthresh)
        
        if par == 'T2map':
            conds = ['control']
        if par == 'func':
            conds = ['all']
        if par == 'cbv' and ID < 56:
            conds=['control']
        if par == 'cbv' and ID > 55:
            conds=['control', 'calc']
            

        for cond in conds:
            ## load parameter map
            if par != 'T2map' and par != 'cmro2':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T2_'+ par +'.nii.gz')
            if par == 'T2map':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_space-T2_'+ par +'.nii.gz')
            if par == 'cmro2' and cond != 'calc':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T2_desc-orig_'+ par +'.nii.gz')
            if par == 'cmro2' and cond == 'calc':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T2_desc-CBV_'+ par +'.nii.gz')
            if par == 'func':
                    par_nii =  os.path.join(dir_deriv, 'func', sub + '_task-'+cond+'_space-T2_filtered_func.nii.gz')
                    
            ##mask parameter map
            out_name = par_nii[:-7] + '_qBmasked.nii.gz'
            par_masked = mask_nii(par_nii, mask_list, out_name)
            

In [ ]:
## mask 1st level ROIs, output from 1st level GLM

for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    dir_deriv = os.path.join(derivatives_dir, sub)
            
    if ID<56:
        ROIs = ['calccontrol', 'controlcalc', 'calcrest','restcalc', 'memcontrol', 'controlmem'] #visuomotor: control>rest
    if ID>55:
        ROIs = ['calccontrol', 'controlcalc']

    mask_list = [ os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_GM_mask.nii.gz'), 
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_T2map_mask.nii.gz'), 
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_R2prime_mask.nii.gz'),
            os.path.join(dir_deriv, 'qmri', sub + '_task-control_space-T2_cbv.nii.gz'),
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_T2Smap_mask.nii.gz'), 
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_oef_mask.nii.gz'), 
              os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_cbf_mask.nii.gz')]


    for ROI in ROIs:
        ROI_nii = os.path.join(dir_deriv, 'func', sub + '_1stlevel_'+ROI+'_space-T2.nii.gz')
                    
        ##mask parameter map
        out_name = ROI_nii[:-7] + '_qBmasked.nii.gz'
        par_masked = mask_nii(ROI_nii, mask_list, out_name)

In [ ]:
## turn 1st level ROIs into lowres space and mask there!

for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    dir_deriv = os.path.join(derivatives_dir, sub)
            
    if ID<56:
        ROIs = ['calccontrol', 'controlcalc', 'calcrest','restcalc', 'memcontrol', 'controlmem'] #visuomotor: control>rest
    if ID>55:
        ROIs = ['calccontrol', 'controlcalc']

    mask_list = [ os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_GM_mask.nii.gz'), 
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_T2map_mask.nii.gz'), 
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_R2prime_mask.nii.gz'),
            os.path.join(dir_deriv, 'qmri', sub + '_task-control_space-T2_cbv.nii.gz'),
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_T2Smap_mask.nii.gz'), 
             os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_oef_mask.nii.gz'), 
              os.path.join(dir_deriv, 'qmri',  sub + '_space-T2_cbf_mask.nii.gz')]


    for ROI in ROIs:
        ROI_nii = os.path.join(dir_deriv, 'func', sub + '_1stlevel_'+ROI+'_space-T2.nii.gz')
                    
        ##mask parameter map
        out_name = ROI_nii[:-7] + '_qBmasked.nii.gz'
        par_masked = mask_nii(ROI_nii, mask_list, out_name)